<a href="https://colab.research.google.com/github/rkrissada/100DayOfMLCode/blob/master/day_067_superstore_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Superstore Forecasting

We will demonstrate different approaches for forecasting retail sales time series.